# Under the hood

The methodology extends Auclert et al. (2022, ECMA) to solve the system fully nonlinearly. 
Most heterogeneous agent models can be expressed in terms of the three functions $V$, $D$ and $F$:

$$
\begin{align}
v_t =& V(x_{t-1}, x_t, x_{t+1}, v_{t+1})\\
d_t =& D(x_{t-1}, x_t, x_{t+1}, v_t, d_{t-1})\\
0 =& f(x_{t-1}, x_t, x_{t+1}, d_t, v_t)
\end{align}
$$

where $x_t$ is the time-$t$ vector of _aggregate_ variables, with each aggregate variables $x_{i,t}$ indexed over $i \in 1,2, \dots, N$. $v_t$ containts the idiosyncratic agents decision functions and $d_t$ is the distribution across agents.

## The steady state

Let the steady state values be denoted by a bar, i.e. $\bar{x}$, $\bar{d}$ and $\bar{v}$. The steady state is often not unique. In this case it is necessary to fix some variables ex-ante, i.e. to given values. Denote the set of indices of the values to be fixed as $K$ and the fixed values as $\hat{x}$. Further, let $b(x)$ be a function that sets $x_K=\hat{x}$, i.e.
$$
b(x) = \{x_{\not\in K} \land \hat{x}\}.
$$

The steady state then satisfies
$$
\begin{align}
\bar{v} =& V(\bar{x}, \bar{x}, \bar{x}, \bar{v})\\
\bar{d} =& D(\bar{x}, \bar{x}, \bar{x}, \bar{v}, \bar{d})\\
0 =& f(\bar{x}, \bar{x}, \bar{x}, \bar{d}, \bar{v})\\
\bar{x} =& b(\bar{x})
\end{align}
$$

For a given guess on $\bar{x}$, the first equation can be solved for $\bar{v}$ via backwards iteration. Denote the function that solves this as $\bar{v}=\bar{V}(\bar{x})$.
Given $\bar{x}$ and $\bar{v}$, the second equation can be solved via forward iteration (or via eigenvector, but eigenvectors are not yet available for automatic differentiation). Denote this solver as $\bar{d} = \bar{D}(\bar{x},\bar{v})$. Define $\bar{f}$ equivalently.

Combining those, $\bar{x}$ must satisfy a function $H$ defined by
$$
H(\bar{x}) = \bar{f}\left(b(\bar{x}), \bar{D}\left(b(\bar{x}), \bar{V}(b(\bar{x}))\right), \bar{V}(b(\bar{x}))\right) = 0.
$$

We can calculate the Jacobian of $H$ using automatic differentiation and, starting with some guess $X^j$ on $\bar{x}$, we can iterate on $H$ using a Newton method. During iteration we use the Moore–Penrose inverse for the inversion of the Jacobian since $\bar{f}$ may not have full rank, in which case it is necessary that $K \neq \{\}$, from which it follows that $b(x)$ also does not have full rank.


## Dynamic solution

Let, for the sake of generality, letters with a tilde denote the complete expected time series/sequence of a variable up to some distant point $T$ in the future, e.g $\tilde{x} = \{x_{t}\}_{t=0}^T$. The above system can be written as

$$
\begin{align}
v_t =& V(\tilde{x}, v_{t+1})\\
d_t =& D(\tilde{x}, \tilde{v}, d_{t-1})\\
0 =& F(\tilde{x}, \tilde{d}, \tilde{v})
\end{align}
$$

with $F=\{f(x_{t-1},x_t,x_{t+1}, d_t, v_t)\}_{t=1}^T$. 

$V$ often takes the form of a value function. Note that the above assumes that $V$ is independent of the distribution at any point in time. Assuming that after $T$ we are back in the steady state, the sequence $\tilde{v}$ can be found by starting with

$$
\begin{align}
v_T =& V(\tilde{x}, \bar{v})\\
v_{T-1} =& V(\tilde{x}, v_T)\\
\dots =& V(\tilde{x}, v_{T-1})
\end{align}
$$

and iterating backwards until we arrive at $v_0$. This yields the sequence $\tilde{v}$.

Now taking $v_0$ as given and assuming we are also in the steady state in $t=0$, we can start with $d_{-1}=\bar{d}$ and iterate $D$ forward:

$$
\begin{align}
d_0 =& D(\tilde{x}, \tilde{v}, \bar{d})\\
d_1 =& D(\tilde{x}, \tilde{v}, d_0)\\
\dots =& D(\tilde{x}, \tilde{v}, d_2)
\end{align}
$$

until we arrive at $d_T$. This yields $\tilde{d}$.

Denote the function that generates $\tilde{v}$ from $\tilde{x}$ as $\hat{V}$, and a function that generates $\tilde{d}$ from $\tilde{x}$ and $\tilde{v}$ as $\hat{D}$. Then we can define

$$
G(\tilde{x}) = F\left(\tilde{x}, \hat{D}\left(\tilde{x}, \hat{V}(\tilde{x})\right), \hat{V}(\tilde{x})\right)
$$

as a function that only depends on the sequence $\tilde{x}$.

We are looking for a sequence $\tilde{x}$ of aggregate variables such that $G(\tilde{x})=0$. Given that the jacobian $J_G$ of $G(\cdot)$ is available via automatic differentiation, we can then use a Newton method to solve for $\tilde{x}$ directly without having to explicitely keep track of any distribution variable. Note that (a) $J_G$ is sparse, so we can make use of sparse linear algebra when solving for the next Newton guess. Also note (b) that not all entries of $f$ depend on aggregated heterogeneous inputs, which can be used to calculate most of the individual blocks of $G_J$ directly from $f$ instead of having to diffentiate the complete function $F$, which is computationally relatively expensive.

## Implementation

The above functions are written dynamically during parsing (in `parsing.py`).

In [1]:
import econpizza as ep
from econpizza import example_hank

In [2]:
mod = ep.load(example_hank)

(load:) Parsing done.


The model instance is a dictionary, containing all the informations of the model. For instance, it contains the parsed functions as strings:

In [3]:
mod['func_strings'].keys()

dict_keys(['func_backw', 'func_stst_dist', 'func_dist', 'func_pre_stst', 'func_eqns'])

The function `func_backw` corresponds to function $V(\cdot)$ above, `func_stst_dist` is $\bar{D}(\cdot)$, `func_dist` is $D(\cdot)$, `func_pre_stst` corresponds to $b(\cdot)$, and `func_eqns` is $f(\cdot)$. $F$ and $\bar{f}$ are created dynamically in `stacking.py` and `steady_state.py`.

Lets inspect $f$:

In [4]:
print(mod['func_strings']['func_eqns'])

def func_eqns(XLag, X, XPrime, XSS, shocks, pars, distributions=[], decisions_outputs=[]):
        
        
 (BLag, betaLag, CLag, DivLag, LLag, piLag, rLag, rnLag, rstarLag, TaxLag, vphiLag, wLag, YLag, YprodLag, ZLag, ) = XLag
        
 (B, beta, C, Div, L, pi, r, rn, rstar, Tax, vphi, w, Y, Yprod, Z, ) = X
        
 (BPrime, betaPrime, CPrime, DivPrime, LPrime, piPrime, rPrime, rnPrime, rstarPrime, TaxPrime, vphiPrime, wPrime, YPrime, YprodPrime, ZPrime, ) = XPrime
        
 (BSS, betaSS, CSS, DivSS, LSS, piSS, rSS, rnSS, rstarSS, TaxSS, vphiSS, wSS, YSS, YprodSS, ZSS, ) = XSS
        
 (eis, frisch, rho_e, sd_e, mu, theta, psi, phi_pi, phi_y, rho, rho_beta, rho_rstar, rho_Z, ) = pars
        
 () = shocks
        
 (D, ) = distributions
        
 (a, c, ne, ) = decisions_outputs
        
 A = jnp.sum(D*a, axis=(0,1))
 NE = jnp.sum(D*ne, axis=(0,1))
 Caggr = jnp.sum(D*c, axis=(0,1))
        
 root_container0 = L - ( Yprod / Z)
 root_container1 = Div - ( - w * L + (1 - psi*(pi/piSS 

This function is then automatically compiled and the callable can be found in `model['context']`:

In [5]:
mod['context']['func_eqns']

<function econpizza.parsing.func_eqns(XLag, X, XPrime, XSS, shocks, pars, distributions=[], decisions_outputs=[])>